<a href="https://colab.research.google.com/github/daonly/personal/blob/main/%5BConf%5D_physio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install neurokit2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 14.3 MB/s eta 0:00:00


In [3]:
pip install bioread

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.3/54.3 kB 1.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13705 sha256=583bdf05a85062ec6846872f805934482bb253f8b3352c9679d9b43b3635b552
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [2]:
### preparation
import neurokit2 as nk
import bioread as br
import os
import pandas as pd
import numpy as np

### set path
from google.colab import drive

drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/Res/Conformity/data')
datapath = r'/content/drive/MyDrive/Res/Conformity/data'
savepath = r'/content/drive/MyDrive/Res/Conformity/analyses'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
### import and set raw data: range 설정 필!

tasklist = ['HRV', 'HCT', 'BlockI', 'BlockII']

for i in range(3,11):

  txtpath = datapath+f'/sub{i}.txt'
  data = pd.read_csv(txtpath, delimiter='\t', low_memory=False)
  data = data.iloc[1:]; data = data.iloc[:, :-1]

  signal = []
  for j in range(1, len(data)):
    if data['CH25'][j] == '0' and data['CH25'][j+1] == '5':
      signal.append(j+1)

  for j in range(4):
    slicedf = data.iloc[signal[2*j]:signal[2*j+1]]
    savefile = datapath+f'/sub{i}_{tasklist[j]}.csv'
    slicedf.to_csv(savefile, index=True)


In [ ]:
### HRV: range 설정 필!

hrvdf = pd.DataFrame()

for i in range(1,2):

  ## SubID
  SubID = str(i)
  hrvdf.loc[0, 1] = SubID
  hrvdf.columns = ['SubID']

  ## Compute time-domain indices of HRV
  hrvfile = datapath+f'/sub{SubID}_HRV.csv'
  data = pd.read_csv(hrvfile)
  peaks, info = nk.ecg_peaks(data['CH1'], sampling_rate=2000)
  hrvtime = nk.hrv_time(peaks, sampling_rate=2000, show=False)
  hrvdf = pd.concat([hrvdf, hrvtime], axis=1, ignore_index=False)

  ## Compute frequency-domain indices of HRV
  hrvfreq = nk.hrv_frequency(peaks, sampling_rate=2000, show=False)
  hrvdf = pd.concat([hrvdf, hrvfreq], axis=1, ignore_index=False)

  ## Compute nonlinear indices of HRV
  hrvnonlin = nk.hrv_nonlinear(peaks, sampling_rate=2000, show=False)
  hrvdf = pd.concat([hrvdf, hrvnonlin], axis=1, ignore_index=False)

for i in range(2,11):
  hrvdf1 = pd.DataFrame()

  ## SubID
  SubID = str(i)
  hrvdf1.loc[0, 1] = SubID
  hrvdf1.columns = ['SubID']

  ## Compute time-domain indices of HRV
  hrvfile = datapath+f'/sub{SubID}_HRV.csv'
  data = pd.read_csv(hrvfile)
  peaks, info = nk.ecg_peaks(data['CH1'], sampling_rate=2000)
  hrvtime = nk.hrv_time(peaks, sampling_rate=2000, show=False)
  hrvdf1 = pd.concat([hrvdf1, hrvtime], axis=1, ignore_index=False)

  ## Compute frequency-domain indices of HRV
  hrvfreq = nk.hrv_frequency(peaks, sampling_rate=2000, show=False)
  hrvdf1 = pd.concat([hrvdf1, hrvfreq], axis=1, ignore_index=False)

  ## Compute nonlinear indices of HRV
  hrvnonlin = nk.hrv_nonlinear(peaks, sampling_rate=2000, show=False)
  hrvdf1 = pd.concat([hrvdf1, hrvnonlin], axis=1, ignore_index=False)
  hrvdf = pd.concat([hrvdf, hrvdf1], ignore_index=True)

savefile = savepath+f'/HRVData.xlsx'
hrvdf.to_excel(savefile, index=False)

hrvdf

,SubID,HRV_MeanNN,HRV_SDNN,HRV_SDANN1,HRV_SDNNI1,HRV_SDANN2,HRV_SDNNI2,HRV_SDANN5,HRV_SDNNI5,HRV_RMSSD,...,HRV_SampEn,HRV_ShanEn,HRV_FuzzyEn,HRV_MSEn,HRV_CMSEn,HRV_RCMSEn,HRV_CD,HRV_HFD,HRV_KFD,HRV_LZC
0,1,704.829412,37.167416,20.923547,30.776604,9.512784,34.715052,NaN,NaN,18.568397,...,1.384070,7.630802,0.901404,1.517830,1.361560,1.991713,1.611565,1.684847,3.061349,0.687803
1,2,700.049708,36.119660,15.096099,32.492288,13.711461,34.160762,NaN,NaN,24.778166,...,1.483699,7.608257,1.126199,1.349342,1.376736,1.992002,1.728019,1.705074,2.683472,0.824819
2,3,886.620988,83.796091,23.131478,78.479324,16.749757,81.757794,NaN,NaN,34.623952,...,1.059018,8.038463,0.730788,1.444858,1.361401,1.889995,1.504829,1.488213,2.541480,0.684387
3,4,704.956863,51.068255,17.453069,48.402857,17.216901,49.102515,NaN,NaN,29.061220,...,1.152931,7.947208,0.892811,1.505926,1.357808,2.091503,1.681108,1.645246,2.522077,0.670168
4,5,820.605263,39.603269,24.283000,32.719957,18.531243,36.471350,NaN,NaN,25.785448,...,1.486581,7.586093,1.141741,1.432577,1.316373,2.046386,1.738970,1.878560,3.380390,0.822954
5,6,705.851670,45.216584,10.448081,42.926266,8.280830,43.468636,NaN,NaN,33.161329,...,1.620526,7.779718,1.167355,1.486505,1.405381,2.164505,1.743876,1.804230,3.089997,0.830258
6,7,696.750485,52.799584,14.953987,50.645403,15.891698,50.762706,NaN,NaN,25.927887,...,0.956454,7.951839,0.850970,1.302533,1.359973,2.033037,1.722000,1.537693,2.762449,0.612223
7,8,528.111765,15.220108,8.883345,12.476236,7.765510,13.862919,NaN,NaN,6.211657,...,1.154933,6.767170,0.770826,1.345796,1.285120,1.971745,1.539919,1.620791,2.251613,0.511982
8,9,930.938961,88.148878,49.241471,74.552842,53.059180,76.713825,NaN,NaN,70.988312,...,1.705145,8.064524,1.312633,1.300915,1.364072,2.032078,1.853590,1.950506,3.706693,0.847717
9,10,633.235035,25.170000,18.800230,17.714289,14.351009,21.692130,NaN,NaN,8.583391,...,0.958992,7.197297,0.698204,1.293942,1.267485,1.847445,1.449217,1.582408,2.254405,0.579914


In [18]:
### HCT: range 설정 필!

hctdf = pd.DataFrame()
hctdf.loc[1, 1] = '1'
hctdf.columns = ['SubID']

for i in range(1,11):

  ## SubID
  SubID = str(i)
  hctdf.loc[i, 'SubID'] = SubID

  ## Extract basic features
  hctfile = datapath+f'/sub{SubID}_HCT.csv'
  data = pd.read_csv(hctfile)

  processed_df, info = nk.bio_process(ecg=data['CH1'], rsp=data['CH3'], sampling_rate=2000)

  ## Set epochs
  hctpath = datapath+f'/sub{i}_HCT.csv'
  data = pd.read_csv(hctpath, low_memory=False)

  startsignal = []; endsignal = []
  for j in range(len(data)-1):
    if data['CH21'][j] == 0 and data['CH21'][j+1] == 5:
      startsignal.append(j+1)
    elif data['CH21'][j] == 5 and data['CH21'][j+1] == 0:
      endsignal.append(j+1)

  ## count R-peaks in each epoch
  hctdf1 = pd.DataFrame()
  for j in range(6):
    slicedf = processed_df.iloc[startsignal[2*j]:endsignal[2*j]]
    rpeaks = slicedf['ECG_R_Peaks'].sum()
    hctdf1.loc[j+1, 1] = rpeaks

  ## calculate stanIAcc & alteIAcc
  rawcardf = pd.read_excel(datapath+f'/HCT/Cardiac-{i}.xls', header=1)
  rawcardf = rawcardf[['HC.RESP','HCCR.Slider1.Value']].iloc[2:8]
  rawcardf.index = range(1,7)
  hctdf1 = pd.concat([hctdf1, rawcardf], axis=1, ignore_index=False)
  hctdf1['stanIAcc'] = 1 - abs(hctdf1['HC.RESP']-hctdf1[1])/hctdf1['HC.RESP']
  hctdf1['alteIAcc'] = 1 - abs(hctdf1['HC.RESP']-hctdf1[1])*2/(hctdf1['HC.RESP']+hctdf1[1])
  hctdf.loc[i, 'stanIAcc'] = hctdf1['stanIAcc'].mean()
  hctdf.loc[i, 'alteIAcc'] = hctdf1['alteIAcc'].mean()

  ##

,SubID,stanIAcc,alteIAcc
1,1,3.974708,2.174245
2,2,1.846521,1.579088
3,3,1.126184,1.112505
4,4,1.256433,1.220513
5,5,1.746168,1.519062
6,6,2.134554,1.607507
7,7,1.046364,1.043749
8,8,2.822019,1.915199
9,9,0.900592,0.894195
10,10,2.095673,1.639722


In [24]:
hctdf = hctdf[['stanIAcc', 'alteIAcc', 'IAw']]
bhvData = datapath+f'/bhvData.xlsx'
with pd.ExcelWriter(bhvData, engine='openpyxl', mode='a', if_sheet_exists='overlay') as writer:
  hctdf.to_excel(writer, sheet_name='Sheet1', startrow=1, startcol=58, index=False, header=False)

In [ ]:
  rawcardf = pd.read_excel(datapath+f'/HCT/Cardiac-1.xls', header=1)
  rawcardf = rawcardf[['HC.RESP','HCCR.Slider1.Value']].iloc[2:8]
  rawcardf.index = range(1,7)
  hctdf1 = pd.concat([hctdf1, rawcardf], axis=1, ignore_index=False)
  hctdf1['stanIAcc'] = 1 - abs(hctdf1['HC.RESP']-hctdf1[1])/hctdf1['HC.RESP']
  hctdf1['alteIAcc'] = 1 - abs(hctdf1['HC.RESP']-hctdf1[1])*2/(hctdf1['HC.RESP']+hctdf1[1])
  hctdf1['IAw'] = hctdf1[1].corr(hctdf1['HCCR.Slider1.Value'])

  hctdf1

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-26-67b888c3b1a4>", line 5, in <cell line: 5>
    hctdf1['stanIAcc'] = 1 - abs(hctdf1['HC.RESP']-hctdf1[1])/hctdf1['HC.RESP']
  File "/usr/local/lib/python3.10/dist-packages/pandas/core/ops/common.py", line 72, in new_method
    return method(self, other)
  File "/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py", line 110, in __sub__
    return self._arith_method(other, operator.sub)
  File "/usr/local/lib/python3.10/dist-packages/pandas/core/frame.py", line 7592, in _arith_method
    self, other = ops.align_method_FRAME(self, other, axis, flex=True, level=None)
  File "/usr/local/lib/python3.10/dist-packages/pandas/core/ops/__init__.py", line 307, in align_method_FRAME
    left, right = left.align(
  File "/usr/local/lib/python3.10/dist-packages/pandas/cor